## Importing all necessary libraries

In [34]:
import os
import pandas as pd
import time
import json
import re
import numpy as np
from collections import defaultdict

# Importing our modules
import crawler
import parser

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


# 1. Data Collection

## 1.1-1.2 Crawling the website to get all the listed restaurants

In [ ]:
crawler.begin()

['https://guide.michelin.com/en/restaurants', 'https://guide.michelin.com/en/hotels', 'https://guide.michelin.com/en/articles', 'https://guide.michelin.com/en/restaurantlist', 'https://guide.michelin.com/en/restaurants', 'https://guide.michelin.com/en/hotels', 'https://guide.michelin.com/en/best-of', 'https://guide.michelin.com/en/articles', 'https://guide.michelin.com/en/restaurantlist', 'https://guide.michelin.comhttps://intercom.help/michelin-guide-contact-us/en/', 'https://guide.michelin.com/en/subscribe', 'https://guide.michelin.comhttps://guide.michelin.com/en/take-advantage-of-the-privileged-partnership-between-the-michelin-guide-thefork-and-tripadvisor', 'https://guide.michelin.comhttps://www.theforkmanager.com/partnership-thefork-michelin#cc=michelin-referral', 'https://guide.michelin.com#', 'https://guide.michelin.com/en/campania/gragnano/restaurant/o-me-o-il-mare', 'https://guide.michelin.com/en/abruzzo/popoli_1845563/restaurant/donevandro', 'https://guide.michelin.com/en/pi

## 1.3 Parsing html files and extracting info and create dataset

In [4]:
restaurant_data = []
folder_path ='restaurants_html'
counter = 1

#Iterate on directories and files
for root, dirs, files in os.walk(folder_path):
    for html_file in files:
        file_path = os.path.join(root, html_file)

        with open(file_path, 'r', encoding='utf-8') as file:
            html_content = file.read()

            restaurant_info = parser.extract_restaurant_info(html_content)

            restaurant_data.append(restaurant_info)

            print(f"Extracted info for {restaurant_info['restaurantName']} number {counter}")
            counter += 1

df_restaurants = pd.DataFrame(restaurant_data)

Extracted info for 20Tre number 1
Extracted info for Alessandro Feo number 2
Extracted info for Ape Vino e Cucina number 3
Extracted info for Charleston number 4
Extracted info for Da Bob Cook Fish number 5
Extracted info for DA_MÓ number 6
Extracted info for Dama number 7
Extracted info for Donevandro number 8
Extracted info for Etra number 9
Extracted info for Il Ristorante Alain Ducasse Napoli number 10
Extracted info for Il Tirabusciò number 11
Extracted info for La Buca number 12
Extracted info for La Trattoria Enrico Bartolini number 13
Extracted info for LoRo number 14
Extracted info for Ménage number 15
Extracted info for O Me O Il Mare number 16
Extracted info for Palazzo Utini number 17
Extracted info for Procaccini number 18
Extracted info for Sa Domu Sarda number 19
Extracted info for Soul & Fish number 20
Extracted info for Bàcaro Il Gusto number 21
Extracted info for Casa Rispoli number 22
Extracted info for Castello di Fighine number 23
Extracted info for Dolada number 2

In [5]:
display(df_restaurants)

,restaurantName,address,city,postalCode,country,priceRange,cuisineType,description,facilitiesServices,creditCards,phoneNumber,website
0,20Tre,via David Chiossone 20 r,Genoa,16123,Italy,€€,"Farmtotable,ModernCuisine",Situated in the heart of Genoa’s historic cent...,[Air conditioning],"[Amex, Dinersclub, Mastercard, Visa]",+39 010 247 6191,https://www.ristorante20tregenova.it/
1,Alessandro Feo,via Angelo Lista 24,Marina di Casal Velino,84040,Italy,€€,"Campanian,Seafood",In a beautiful stone-vaulted building (an old ...,[],"[Amex, Dinersclub, Mastercard, Visa]",+39 328 893 7083,https://www.alessandrofeoristorante.it/
2,Ape Vino e Cucina,Piazza Risorgimento 3,Alba,12051,Italy,€€,"Piedmontese,Contemporary",This attractive restaurant in the heart of Alb...,"[Air conditioning, Terrace, Wheelchair access]","[Amex, Dinersclub, Maestrocard, Mastercard, Visa]",+39 0173 363453,https://www.apewinebar.it/alba/
3,Charleston,via Generale Magliocco 19,Palermo,90141,Italy,€€€€,"ModernCuisine,Creative","Before it became famous in Mondello, the renow...","[Air conditioning, Counter dining, Terrace, Wh...","[Amex, Mastercard, Visa]",+39 091 450171,https://casacharleston.net/
4,Da Bob Cook Fish,largo Parsano vecchio 16,Sorrento,80067,Italy,€€,Seafood,Working in partnership with the nearby fishmon...,"[Air conditioning, Terrace]","[Amex, Dinersclub, Mastercard, Visa]",+39 081 1778 3873,https://www.dabobcookfish.com/
...,...,...,...,...,...,...,...,...,...,...,...,...
1977,Shiroya,via dei Baullari 147,Rome,00186,Italy,€€,"Japanese,Asian",One of the most popular restaurants in the his...,"[Air conditioning, Terrace]","[Amex, Mastercard, Visa]",+39 06 6476 0753,https://www.shiroya.it
1978,Sotto l'Arco,via Aretusi 5,Bologna,40132,Italy,€€€,"Italian,Creative",Villa Aretusi is a pleasant 17C villa surround...,"[Air conditioning, Car park, Garden or park, T...","[Mastercard, Visa]",+39 051 619 9848,https://www.villa-aretusi.it/ristorante-sotto-...
1979,Umami,Via Ugo Secondo Partigiano 1,Badalucco,18010,Italy,€€,ModernCuisine,A young chef with experience in renowned resta...,"[Terrace, Wheelchair access]","[Amex, Mastercard, Visa]",+39 331 338 6005,https://www.umamirestaurant.it/
1980,Visione Restaurant and Living,Strada Nicolini Basso 34,loc. Tre Stelle,Barbaresco,12050,€€€,"Contemporary,Piedmontese","At this restaurant, new, young and enthusiasti...","[Air conditioning, Car park]","[Amex, Maestrocard, Mastercard, Visa]",+39 328 134 0218,https://www.ristorantevisione.it


In [5]:
print(df_restaurants.isna().sum())  

restaurantName        0
address               0
city                  0
postalCode            0
country               0
priceRange            0
cuisineType           0
description           0
facilitiesServices    0
creditCards           0
phoneNumber           0
website               0
dtype: int64


In [6]:
print(df_restaurants.shape)

(1982, 12)


In [29]:
for i, row in df_restaurants.iterrows():
    # Define the file name using the index
    file_name = f"restaurant_{i}.tsv"

    # Prepare row data as a single line with tab-separated values
    content =  f"{row['restaurantName']}\t{row['address']}\t{row['city']}\t{row['postalCode']}\t{row['country']}\t{row['priceRange']}\t{row['cuisineType']}\t{row['description']}\t{row['facilitiesServices']}\t{row['creditCards']}\t{row['phoneNumber']}\t{row['website']}\n"

    subfolder = f"files_tsv"
    file_path = os.path.join(subfolder, file_name)

    # Check if the subfolder exists, create it if it doesn't
    if not os.path.exists(subfolder):
        os.makedirs(subfolder)
    # Write the row data to the .tsv file
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(content)

    print(f"Created file: {file_name}")

Created file: restaurant_0.tsv
Created file: restaurant_1.tsv
Created file: restaurant_2.tsv
Created file: restaurant_3.tsv
Created file: restaurant_4.tsv
Created file: restaurant_5.tsv
Created file: restaurant_6.tsv
Created file: restaurant_7.tsv
Created file: restaurant_8.tsv
Created file: restaurant_9.tsv
Created file: restaurant_10.tsv
Created file: restaurant_11.tsv
Created file: restaurant_12.tsv
Created file: restaurant_13.tsv
Created file: restaurant_14.tsv
Created file: restaurant_15.tsv
Created file: restaurant_16.tsv
Created file: restaurant_17.tsv
Created file: restaurant_18.tsv
Created file: restaurant_19.tsv
Created file: restaurant_20.tsv
Created file: restaurant_21.tsv
Created file: restaurant_22.tsv
Created file: restaurant_23.tsv
Created file: restaurant_24.tsv
Created file: restaurant_25.tsv
Created file: restaurant_26.tsv
Created file: restaurant_27.tsv
Created file: restaurant_28.tsv
Created file: restaurant_29.tsv
Created file: restaurant_30.tsv
Created file: rest

# 2. Search engines

## 2.0 Preprocessing the text

In [2]:
### Cleaning and preparing text of description column in df

# Function to remove stopwords, punctuation and apply stemming on text  
def clean_desc(desc):
    # Get english stopwords
    stop_words = set(stopwords.words('english'))

    # Applying other necessary cleaning: 1 - lower casing
    desc = desc.lower()

    # Applying other necessary cleaning: 2 - removing special characters, non-alphanumerics 
    desc = re.sub(r'http\S+|www\S+|https\S+', '', desc)  # Remove URLs
    desc = re.sub(r'\S+@\S+', '', desc)  # Remove email addresses
    desc = re.sub(r'[^A-Za-z0-9\s]', '', desc)  # Remove non-alphanumeric characters

    words = word_tokenize(desc)

    stemmer = PorterStemmer()

    remaining = [
        stemmer.stem(word) # Apply stemming at last / same order provided in HW requirements
        for word in words
        if word.lower() not in stop_words and word not in string.punctuation]
    
    # Applying other necessary cleaning: 3 - removing extra spaces 
    cleaned_text = ' '.join(remaining) 
    remaining = re.sub(r'\s+', ' ', cleaned_text).strip()
    return cleaned_text

In [6]:
# apply function to description column 
df_restaurants['cleaned_desc'] = df_restaurants['description'].apply(clean_desc)

In [6]:
df_restaurants

,restaurantName,address,city,postalCode,country,priceRange,cuisineType,description,facilitiesServices,creditCards,phoneNumber,website,cleaned_desc
0,L'Acciuga,via Settevalli 217,Perugia,06128,Italy,€€€,"Contemporary,International",You would never guess that there was a gourmet...,"[Air conditioning, Interesting wine list, Terr...","[Amex, Unionpay, Dinersclub, Discover, Jcb, Ma...",+39 339 263 2591,https://www.lacciuga.net/,would never guess gourmet restaur tuck away co...
1,Jamantè,via San Vito 97,Polignano a Mare,70044,Italy,€€€,ModernCuisine,Located not far from the historic centre and t...,"[Air conditioning, Terrace]","[Amex, Dinersclub, Mastercard, Visa]",+39 351 628 7773,https://www.jamanteristorante.com,locat far histor centr picturesqu lama monachi...
2,Cucine Nervi,corso Vercelli 117,Gattinara,13045,Italy,€€€,ModernCuisine,Housed within the Cantine Nervi in the town ce...,[Air conditioning],"[Mastercard, Visa]",+39 333 182 4123,https://cucinenervi.com,hous within cantin nervi town centr contempora...
3,Lokanda Devetak,via Brezici 22,Savogna d'Isonzo,34070,Italy,€€,"RegionalCuisine,TraditionalCuisine",Just two kilometres from the Slovenian border ...,"[Air conditioning, Car park, Garden or park, I...","[Amex, Mastercard, Visa]",+39 0481 882488,https://www.devetak.com/,two kilometr slovenian border crow fli restaur...
4,Felix Lo Basso home & restaurant,via Carlo Goldoni 36,Milan,20129,Italy,€€€€,"ItalianContemporary,Creative",Brilliant chef Felix Lo Basso’s menu is inspir...,"[Air conditioning, Counter dining, Wheelchair ...","[Amex, Mastercard, Visa]",+39 02 4540 9759,https://www.felixlobassorestaurant.it/,brilliant chef felix lo basso menu inspir nort...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1977,Kleine Flamme,via Cittanuova 31,Vipiteno,39049,Italy,€€€,"Fusion,Creative",In his small open-view kitchen situated in the...,[Terrace],"[Mastercard, Visa]",+39 0472 766065,https://kleineflamme.com/it/,small openview kitchen situat centr restaur ch...
1978,Antica Locanda al Cervo - Landgasthof zum Hirs...,via Schrann 9/c,San Genesio Atesino,39050,Italy,€€,"RegionalCuisine,TraditionalCuisine","Enjoy generous, regional cuisine influenced by...","[Car park, Garden or park, Great view, Terrace...","[Mastercard, Visa]",+39 0471 354195,https://www.hirschenwirt.it/,enjoy gener region cuisin influenc famili invo...
1979,Lapprodo,via Roma 22,Vibo Valentia Marina,89900,Italy,€€€,"Seafood,ClassicCuisine",Definitely one of the most reliable restaurant...,"[Air conditioning, Terrace, Wheelchair access]","[Amex, Unionpay, Dinersclub, Discover, Maestro...",+39 0963 572640,https://www.lapprodo.com/,definit one reliabl restaur area vibo valentia...
1980,MikEle,via Flavio Gioia 1,Maranello,41053,Italy,€€€,Seafood,Located in a residential district on the edge ...,[Air conditioning],"[Amex, Dinersclub, Mastercard, Visa]",+39 0536 941027,https://www.ristorantemikele.com/,locat residenti district edg town unexpect ele...


In [12]:
my_set = set(word for description in df_restaurants['cleaned_desc'] for word in description.split())

In [13]:
import engine

engine.vocabulary_creator(my_set)
engine.inverted_index_creator(df_restaurants)

In [20]:
print("I need your input")
query = input()
query = clean_desc(query)

intersection_set = engine.conjunctive_query(query)

df_toPrint = []

df_toPrint = pd.DataFrame(df_toPrint)

for restaurant in intersection_set:
    filtered_df = df_restaurants.query("restaurantName == @restaurant")
    df_toPrint = pd.concat([df_toPrint, filtered_df])

df_toPrint.reset_index(drop=True, inplace=True)
df_toPrint = df_toPrint.filter(items=['restaurantName', 'address', 'description', 'website'])
df_toPrint = df_toPrint.rename(columns={'restaurantName':'Restaurant Name', 'address':'Address', 'description': 'Description', 'website':'Website'})


display(df_toPrint)


I need your input


,Restaurant Name,Address,Description,Website
0,Flurin,Laubengasse 2,Flurin occupies an old medieval tower in Glore...,https://www.flurin.it
1,Saporium,località Palazzetto 110,Saporium is the new fine-dining restaurant at ...,http://www.saporium.com/it/borgo-santo.pietro/
2,San Michele,via Castello di Fagagna 33,Situated next to the ruins of the old castle a...,http://sanmichele.restaurant
3,San Giorgio,viale Brigate Bisagno 69r,Situated in the city albeit not right in the c...,https://www.ristorantesangiorgiogenova.it/
4,San Giorgio,via Mancini 3,"Situated in the historic centre of Umbertide, ...",https://www.ristorante-sangiorgio.it/
5,Piccolo Lord,corso San Maurizio 69 bis/g,"Professional service in a welcoming, modern re...",https://www.ristorantepiccololord.it/
6,Locanda Solagna,piazza I Novembre 2,Although this restaurant has been in business ...,https://www.locandasolagna.it/
7,Casin del Gamba,via Roccolo Pizzati 1,The journey to get here – a winding road throu...,https://www.casindelgamba.it/
8,Aprudia,largo del Forno 16,"At this restaurant in the historic centre, whe...",http://www.aprudia.com
9,Il Tino,via Monte Cadria 127,Enjoying an attractive location in the Nautilu...,https://www.ristoranteiltino.com/


In [58]:
engine.inverted_tfidf(df_restaurants)

### 2.3  Non funziona

In [ ]:
with open('ranked_search_inverted_index.json', 'r') as file:
    inverted_index = json.load(file)

def ranked_search_with_cosine(query, top_k=5):
    # Step 1: Clean and tokenize the query
    cleaned_query = clean_desc(query)
    
    query_df = pd.DataFrame({'restaurantName': ['query'], 'cleaned_desc': [cleaned_query]})

    query_tfidf = engine.inverted_tfidf(query_df)
    query_vector = np.array(list(query_tfidf.values())).reshape(1, -1)  

    # Creating a dict with each restaurant with its own similarity score 
    similarities = defaultdict(float)

    # Compute the similarity
    for word, word_tfidf in query_tfidf.items():  #For every word in the query
        if word in inverted_index:  #if exist
            #Looking at every restaurant with that word in its description
            for restaurant, tfidf_value in inverted_index[word]:
                similarities[restaurant] += tfidf_value * word_tfidf  
    
    
    ranked_restaurants = sorted(similarities, key=lambda x: x[1], reverse=True)
    df_toPrint = pd.DataFrame()

    # Selecting best K 
    for restaurant, similarity_score in ranked_restaurants[:top_k]:
        filtered_df = df_restaurants.query("restaurantName == @restaurant")
        # Adding similarities value 
        restaurant_details = filtered_df[['restaurantName', 'address', 'description', 'website']].copy()
        restaurant_details['Similarity score'] = similarity_score
        df_toPrint.append(restaurant_details)

        df_toPrint = pd.concat(df_toPrint)

    df_toPrint = pd.concat(df_toPrint)


    # Index reset and viusualization 
    df_toPrint.reset_index(drop=True, inplace=True)
    df_toPrint = df_toPrint.rename(columns={'restaurantName': 'Restaurant Name', 'address': 'Address', 'description': 'Description', 'website': 'Website'})

    return df_toPrint


In [62]:
def vectorize_and_compute_similarity(df_restaurants, query):

    cleaned_query = clean_desc(query)


    tfidf_vectorizer = TfidfVectorizer(stop_words='english')


    all_descriptions = df_restaurants['description'].apply(clean_desc).tolist()
    all_descriptions.insert(0, cleaned_query)  #    query at the first row


    tfidf_matrix = tfidf_vectorizer.fit_transform(all_descriptions)

    query_vector = tfidf_matrix[0]
    restaurant_vectors = tfidf_matrix[1:]

    # compute the similarity
    similarities = cosine_similarity(query_vector, restaurant_vectors)

    
    return similarities.flatten()

def ranked_search_with_cosine(query, top_k=5):
   
    similarities = vectorize_and_compute_similarity(df_restaurants, query)

    # Add the similarity at the dataframe
    df_restaurants['similarity'] = similarities

    # Sorting the restaurant by similarity
    df_sorted = df_restaurants.sort_values(by='similarity', ascending=False).head(top_k)

    return df_sorted[['restaurantName', 'description', 'similarity']]


In [63]:
ranked_search_with_cosine(query="modern seasonal cuisine")

,restaurantName,description,similarity
178,Saur,"In a tiny rural village, this contemporary, al...",0.315431
1159,Razzo,"A quiet restaurant with a relaxed, young and m...",0.270649
1651,La Botte,A modern and welcoming contemporary bistro sit...,0.267966
996,Piccolo Lord,"Professional service in a welcoming, modern re...",0.256811
513,La Valle,A well - run restaurant in a quiet area just o...,0.239203
